## Incremental Data Load from APIs

Let us understand how to load the data incrementally from APIs into target database table in Dynamodb.
* `GET /repositories` takes since to list public repositories beyond the id passed.
* The repositories listed are pre sorted by id and hence we can keep track of last id and run the code.
* Keep in mind that we can get up to approximately 4950 repo details with created_at per hour due to the GitHub rate limit.
* Once we reach the threshold we need to wait for one hour to get new data. For now we will run the code until we reach the limit.
* If we want to run using docker or AWS Lambda functions, we should plan for exit once the limit is reached and then restart the job approximately 1 hour after.
* Either we can get up to 4900 using `GET /repositories` and then invoke `GET /repos/{owner}/{repo}` for those 4900 or we can process 100 at a time.
* We will use the later approach and try to validate it for 300 repositories. 
  * Get 300 repositories using `GET /repositories`.
  * Use `GET /repos/{owner}/{repo}` to get each of the repository details up to 50.
  * Write all the 50 into the database.
  * This approach will generate data continuously at regular intervals in Dynamodb table.

In [1]:
import requests

In [2]:
import json

In [3]:
def list_repos(token, since='333255899', batches=3):
    repos = []
    for i in range(batches):
        print(f'Getting 100 repos from repo id {since} in {i + 1} iteration')
        res = requests.get(
            f'https://api.github.com/repositories?since={since}',
            headers={'Authorization': f'token {token}'}
        )
        repos_listed = json.loads(res.content.decode('utf-8'))
        repos += repos_listed
        since = repos_listed[-1]['id']
    return repos, since

In [4]:
def get_repo_details(owner, name, token):
#     print(f'Getting repo details for {name}')
    repo_details = json.loads(requests.get(
        f'https://api.github.com/repos/{owner}/{name}',
        headers={'Authorization': f'token {token}'}
    ).content.decode('utf-8'))
    return repo_details

In [5]:
def extract_repo_fields(repo_details):
    repo_fields = {
        'id': repo_details['id'],
        'node_id': repo_details['node_id'],
        'name': repo_details['name'],
        'full_name': repo_details['full_name'],
        'owner': {
            'login': repo_details['owner']['login'],
            'id': repo_details['owner']['id'],
            'node_id': repo_details['owner']['node_id'],
            'type': repo_details['owner']['type'],
            'site_admin': repo_details['owner']['site_admin']
        },
        'html_url': repo_details['html_url'],
        'description': repo_details['description'],
        'fork': repo_details['fork'],
        'created_at': repo_details['created_at']
    }
    return repo_fields

In [6]:
def load_repos(repos_details, table, batch_size=50):
    with table.batch_writer() as batch:
        repos_count = len(repos_details)
        for i in range(0, repos_count, batch_size):
            for repo in repos_details[i:i+batch_size]:
                batch.put_item(Item=repo)  

In [7]:
def get_and_load_repos(repos, token, table, batch_size=50):
    repos_details = []
    repos_size = len(repos)
    for idx, repo in enumerate(repos):
        if (idx + 1) % batch_size == 0:
            print(f'Saving {idx + 1} out of {repos_size}')
            load_repos(repos_details, table, batch_size)
            repos_details = []
        try:
            owner = repo['owner']['login']
            name = repo['name']
            repo_details = get_repo_details(owner, name, token)
            repo_fields = extract_repo_fields(repo_details)
            repos_details.append(repo_fields)
        except:
            # We can log the exceptions into a log table to troubleshoot data quality issues
            pass
    load_repos(repos_details, table, batch_size)
    # We can preserve the id of last repo in database so that we can restart
    # after rate limit is reset
    return repos_details

In [8]:
repos, since = list_repos('bd8a9c237cfd84a454a69ab4f68bc799d4d2e08f')

Getting 100 repos from repo id 333255899 in 1 iteration
Getting 100 repos from repo id 333256141 in 2 iteration
Getting 100 repos from repo id 333256393 in 3 iteration


In [9]:
import boto3

In [10]:
dynamodb = boto3.resource('dynamodb')

In [11]:
ghrepos_table = dynamodb.Table('ghrepos')

In [12]:
%%time
repos_details = get_and_load_repos(repos, 'bd8a9c237cfd84a454a69ab4f68bc799d4d2e08f', ghrepos_table)

Saving 50 out of 300
Saving 100 out of 300
Saving 150 out of 300
Saving 200 out of 300
Saving 250 out of 300
Saving 300 out of 300
CPU times: user 5.26 s, sys: 296 ms, total: 5.56 s
Wall time: 2min 12s


* Deleting the old data so that we can reload.

In [4]:
def delete_repos(repos_details, ghrepos_table, batch_size=50):
    with ghrepos_table.batch_writer() as batch:
    
        repos_count = len(repos_details)
        for i in range(0, repos_count, batch_size):
            print(f'Processing from {i} to {i+batch_size}')
            for repo in repos_details[i:i+batch_size]:
                key = {'id': repo['id']}
                batch.delete_item(Key=key)  

In [13]:
rs = ghrepos_table.scan()

In [14]:
len(rs['Items'])

183

In [ ]:
rs['Items'][0]

In [6]:
%%time
delete_repos(rs['Items'], ghrepos_table)

CPU times: user 19 µs, sys: 0 ns, total: 19 µs
Wall time: 21.9 µs
